In [4]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

In [83]:
signal = np.tile([1,0], 100)
samples = []
for n in range(10):
    max_length = 24
    lb = np.random.randint(1, max_length)
    i = np.random.randint(lb, len(signal))
    s = np.concatenate((np.zeros(max_length - lb), signal[i-lb:i]))
    label = signal[i]
    assert len(s) == max_length
    samples.append((s, label))
samples

[(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0.]),
  1),
 (array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0.]),
  1),
 (array([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 1., 0., 1.]),
  0),
 (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 1., 0., 1.]),
  0),
 (array([0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 1., 0., 1.]),
  0),
 (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 1., 0., 1.]),
  0),
 (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 1., 0., 1.]),
  0),
 (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.]),
  0),
 (array([0., 0., 0., 0.,

In [148]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [106]:
decoder_layer = nn.TransformerDecoderLayer(d_model=24, nhead=8).double()
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=1).double()

In [109]:
sample = torch.from_numpy(np.array([s[0] for s in samples])).double().view(-1, 1, 24)
label = torch.from_numpy(np.array([s[1] for s in samples])).double().view(-1, 1, 1)
l = nn.Linear(24, 1).double()
out = l(transformer_decoder(sample, sample))
out

tensor([[[ 0.4799]],

        [[-0.5945]],

        [[-0.0511]],

        [[-0.2811]],

        [[-0.0335]],

        [[-0.6010]],

        [[-0.5402]],

        [[-0.0976]],

        [[-0.6748]],

        [[-0.5100]]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
class PositionalEncoding(nn.module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)


        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [157]:
class GPTime(nn.Module):
    def __init__(self):
        super(GPTime, self).__init__()
        self.pos_encode = PositionalEncoding()
        decoder_layer = nn.TransformerDecoderLayer(d_model=24, nhead=8)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=4)
        self.linear = nn.Linear(24, 1)

    def forward(self, x):
        x = self.decoder(x, x)
        x = F.relu(x)
        x = self.linear(x)
        x = F.relu(x)
        return x

In [158]:
model = GPTime().double()

In [159]:
criterion = nn.MSELoss()
lr = 0.005 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
NUM_EPOCHS = 1000

for epoch in range(1, NUM_EPOCHS + 1):
    data, target = sample, label

    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()
    #print(loss.item())
print(loss.item())   
print(model(data).detach().numpy().flatten(), target.detach().numpy().flatten())
print((model(data).detach().numpy().flatten() - target.detach().numpy().flatten()).sum())

0.0027288108436819562
[0.9892733  0.95533092 0.         0.         0.         0.
 0.         0.         0.         0.86367359] [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
-0.09443756674104309


In [160]:
data

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
          0., 1., 0., 1., 0., 1., 0.]],

        [[0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
          0., 1., 0., 1., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
          1., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
          1., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
          1., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
          1., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [96]:
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)

In [98]:
memory.shape, tgt.shape

(torch.Size([10, 32, 512]), torch.Size([20, 32, 512]))